# Image Feature Extraction via PCA & kPCA

> *Numerical Optimization and Large Scale Linear Algebra*  
> *MSc in Data Science, Department of Informatics*  
> *Athens University of Economics and Business*

---

For this task, we use images from the Yale Face Database B, which contains $5760$ single light source gray-level images of $10$ subjects, each seen under $576$ viewing conditions. We take $51$ images of the first subject and $51$ images of the third subject as the training data, and $13$ images of each of them as testing data. Then all the images are aligned, and each image has $168 \times 192$ pixels. We use the $168 \times 192$ pixel intensities as the original features for each image, thus the original feature vector is $32256$-dimensional. Then we use standard PCA and Gaussian kernel PCA to extract the $9$ most significant features from the training data, and record the eigenvectors. For standard PCA, only the eigenvectors are needed to extract features from testing data. For Gaussian kernel PCA, both the eigenvectors and the training data are needed to extract features from testing data. Note that for standard PCA, there are particular fast algorithms to compute the eigenvectors when the dimensionality is much higher than the number of data points. For kernel PCA, we use a Gaussian kernel with $\sigma = 22546$. For classification, we use the simplest linear classifier.

Parameter selection for kernel PCA directly determines the performance of the algorithm. For Gaussian kernel PCA, the most important parameter is the $\sigma$ in the kernel function defined by $\kappa(x,y)=\exp(-\|x-y\|^{2} \div 2\sigma^{2})$. The Gaussian kernel is a function of the distance $\| x-y \|$ between two vectors $x$ and $y$. Ideally, if we want to separate different classes in the new feature space, then the parameter $\sigma$ shoud be smaller than inter-class distances, and larger than inner-class distances. However, we don't know how many classes are there in the data, thus it is not easy to estimate the inter-class or inner class distances. Alternatively, we can set $\sigma$ to a small value to capture only the neighborhood information of each data point. For this purpose, for each data point $x_{i}$, let the distance from $x_{i}$ to its nearest neighbor be $d_{i}^{NN}$. In the experiments, we use this parameter selection strategy:<br><br>$$\sigma=5mean(d_{i}^{NN}).$$<br>This strategy, in the experiments, ensures that the $\sigma$ is large enough to capture neighboring data points, and is much smaller than inter-class distances. When using different datasets, this strategy may need modifications.

> [***Kernel Principal Component Analysis and its Applications in Face Recognition and Active Shape Models***](https://github.com/sapaladas/msc_data_science/blob/main/q3-numerical_optimization_and_large_scale_linear_algebra/image_feature_extraction_via_pca_kpca/readings/kernel_principal_component_analysis.pdf)  
> *Quan Wang, Rensselaer Polytechnic Institute, 110 Eighth Street, Troy, NY 12180 USA*  
> *arXiv:1207.3538 \[cs.CV\], 2012*

## Libraries 

In [23]:
import numpy as np
import pandas as pd

import h5py
from sklearn.decomposition import PCA, KernelPCA
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import pdist, squareform

from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

## Data

In [24]:
# import data
faces_data = h5py.File('./Data/YaleFaceData.mat', 'r')

# train and test set
x_train, y_train = faces_data['train_x'][:].T, faces_data['train_t'][:]
x_test, y_test = faces_data['test_x'][:].T, faces_data['test_t'][:]

# shapes
print(f'x_train.shape: {x_train.shape}')
print(f'y_train.shape: {y_train.shape}')
print(f'x_test.shape:  {x_test.shape}')
print(f'y_test.shape:  {y_test.shape}')

x_train.shape: (102, 32256)
y_train.shape: (102, 1)
x_test.shape:  (26, 32256)
y_test.shape:  (26, 1)


## PCA

The **principal components** of a set of points in a real coordinate space form a series of $p$ unit vectors. The $i$-th vector represents the direction of a line that optimally fits the data while being orthogonal to the preceding $i-1$ vectors. A best-fitting line, in this context, minimizes the average squared distance from the points to the line. These vectors create an orthonormal basis where individual dimensions of the data are linearly uncorrelated. **Principal Component Analysis (PCA)** involves computing these principal components and utilizing them to transform the data's basis. Often, only the initial principal components are considered, and the rest are disregarded.

##### *Function for PCA*

In [25]:
def pca(x_train, x_test, components_num = 9):
    
    # initialize PCA
    pca = PCA(n_components=components_num, random_state=123)
    
    # fit PCA
    pca.fit(x_train)
    
    # get principal components
    eigenvectors = pca.components_
    
    # data transformation
    x_train_PCA = np.dot(x_train, eigenvectors.T)
    x_test_PCA = np.dot(x_test, eigenvectors.T)
    
    return x_train_PCA, x_test_PCA

# execute function
x_train_pca, x_test_pca = pca(x_train, x_test)

## Gaussian Kernel PCA 

**Kernel Principal Components Analysis (kernel PCA)** is an advancement in multivariate statistics, extending Principal Components Analysis (PCA) to handle non-linear data separability through the application of kernels. The fundamental concept involves projecting non-linearly separable data onto a higher-dimensional space, transforming it into a linearly separable form.

##### *Implementation*

1. Use Gaussian kernel PCA to project the train data onto a lower dimensional space and record the eigenvectors
2. Use Gaussian kernel PCA and the eigenvectors obtained from previous step, to project the test data onto a lower dimensional space

##### *Function for Kernel PCA*

In [26]:
#function to get the Radial Basis Function Kernel
def gaussian_rbf_kernel(x, sigma = 22546):

    D = pdist(x,'sqeuclidean')
    D = squareform(D)
    D = np.where(D<0,0,D)
    K = np.sqrt(D)
    K = K**2
    K = np.exp(-K/(2*sigma**2))
    
    return K

def kernel_PCA(x, sigma = 22546, components_num = 9):
    
    # construct the kernel matrix
    K0 = gaussian_rbf_kernel(x)
    
    # compute the gram matrix
    N = x.shape[0]
    oneN = np.ones((N,N))/N
    K = K0 - np.dot(oneN,K0) - np.dot(K0,oneN) + np.dot(np.dot(oneN,K0),oneN)
    
    # eigenvalue analysis
    eigenvalues, eigenvectors = np.linalg.eigh(K)
    
    # normalization
    norm_eigenvectors = np.sqrt(sum(eigenvectors**2))
    eigenvectors = eigenvectors / np.tile(norm_eigenvectors, (eigenvectors.shape[0], 1))
    
    # dimensionality reduction
    kLargestIdx = np.argsort(eigenvalues)[::-1][:components_num]
    eigenvectors = eigenvectors[:,kLargestIdx]
    x_train_kPCA = np.dot(K0,eigenvectors)
    
    return x_train_kPCA, eigenvectors

# execute function
x_train_kPCA, eigenvectors = kernel_PCA(x_train)

##### *Project the test data to a lower dimensional space*

In [27]:
def rbf_kernel_projection(x_train, x_test, sigma = 22546):

    D = pdist(np.concatenate([x_train,x_test]), 'sqeuclidean')
    D = squareform(D)
    D = np.where(D<0,0,D)
    K = np.sqrt(D)
    N = x_train.shape[0]
    K = K[N:,:N]
    K = K**2
    K = np.exp(-K/(2*sigma**2))
    
    return K

def kPCA_projection(x_train, x_test, eigenvectors, sigma = 22546):

    # construct the kernel matrix
    K = rbf_kernel_projection(x_train, x_test, sigma)
    
    # dimensionality reduction
    x_test_kPCA = np.dot(K,eigenvectors)
    
    return x_test_kPCA

# execute function
x_test_kPCA = kPCA_projection(x_train, x_test, eigenvectors)

## Results 

In [29]:
def get_error_rates(model, x, y, x_pred, y_true):
    # initialize model
    model = LinearRegression()
    # fit the model in the data
    model.fit(x, y.ravel())
    # make predictions
    predictions = model.predict(x_pred)
    # translate prediction
    predictions = np.where(predictions > 0, 1, -1)
    # compute error rate
    error_rate = 1 - accuracy_score(y_true, predictions)
    return error_rate

# PCA train
error_rate = get_error_rates(LinearRegression(), x_train_pca, y_train, x_train_pca, y_train)
print(f'PCA train error: {round(error_rate*100,2)}%')

# PCA test
error_rate = get_error_rates(LinearRegression(), x_train_pca, y_train, x_test_pca, y_test)
print(f'PCA  test error: {round(error_rate*100,2)}%\n')

# Kernel PCA train
error_rate = get_error_rates(LinearRegression(), x_train_kPCA, y_train, x_train_kPCA, y_train)
print(f'Kernel PCA train error: {round(error_rate*100,2)}%')

# Kernel PCA test
error_rate = get_error_rates(LinearRegression(), x_train_kPCA, y_train, x_test_kPCA, y_test)
print(f'Kernel PCA  test error: {round(error_rate*100,2)}%')

PCA train error: 8.82%
PCA  test error: 23.08%

Kernel PCA train error: 6.86%
Kernel PCA  test error: 11.54%
